### Imports

In [1]:
import numpy as np
import pandas

### Script

In [2]:
drives_df = pandas.read_csv("../../data/raw/drives.csv")

In [3]:
# Leo DiPerna (dipernalz)
# game time data cleaning

# fill in missing quarters and convert column type to int
for i in range(len(drives_df)):
    if np.isnan(drives_df.loc[i, 'quarter']):
        drives_df.loc[i, 'quarter'] = drives_df.loc[i - 1, 'quarter']
drives_df['quarter'] = drives_df['quarter'].astype(int)

# add time_left column, which is game time left in seconds
# time_left is negative if game goes to overtime
drives_df = drives_df.assign(time_left = 0)
for i in range(len(drives_df)):
    mins, secs = drives_df.loc[i, 'time'].split(':')
    drives_df.loc[i, 'time_left'] = 60*(60 - 15*(drives_df.loc[i, 'quarter'])) + 60*int(mins) + int(secs)

In [4]:
# yard line data cleaning

drives_df = drives_df.assign(yard_line = 0)
for i, current_row in drives_df.iterrows():
    team_id = list()
    
    los = current_row['los']
    team_id.append(str(current_row['team_id']))
    
    if ('KC' in team_id):
        team_id.append('KAN')
    elif ('NO' in team_id):
        team_id.append('NOR')
    elif ('TB' in team_id):
        team_id.append('TAM')
    elif ('SF' in team_id):
        team_id.append('SFO')
    elif ('LAR' in team_id):
        team_id.append('STL')
    elif ('GB' in team_id):
        team_id.append('GNB')
    elif ('NE' in team_id):
        team_id.append('NWE')
    elif ('LV' in team_id):
        team_id.append('OAK')
    elif ('LAC' in team_id):
        team_id.append('SDG')
    
    if pandas.isna(los):
        drives_df.loc[i, 'yard_line'] = 0
    else:
        los_team_id, los_yards = los.split()
        if (los_team_id in team_id):
            drives_df.loc[i, 'yard_line'] = int(los_yards)
        elif (los_team_id not in team_id):
            drives_df.loc[i, 'yard_line'] = 100 - int(los_yards)   
    team_id.clear()

In [5]:
drives_df.to_csv("p_drives.csv", index = False)